In [1]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow

In [2]:
# Env vars
torch.use_deterministic_algorithms(True) # to ensure reproduceability
ROOT_DIR = "/mnist"
RAND_SEED = 0
DEVICE = "cpu"

# training hyperparameters
NUM_CLASSES = 10
EPOCHS = 10
#FOURIER_ORDER = 10
LEARNING_RATE = 1e-3
BATCH_SIZE = 500
NUM_TRAIN_BATCHES = 60000 // BATCH_SIZE
NUM_VAL_BATCHES = 10000 // BATCH_SIZE
loss_fn = nn.CrossEntropyLoss()


# function to ensure deterministic worker re-seeding for reproduceability
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

Section 1: Fourier Descriptors through linear classifier

In [8]:
# mlp taking array of normalized fourier descriptors
class LinearClassifierFourier(nn.Module):
    def __init__(self):
        super(LinearClassifierFourier, self).__init__()
        self.flatten = nn.Flatten()
        self.mlp = nn.Sequential(
        nn.Linear(FOURIER_ORDER*4, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,NUM_CLASSES))

    def forward(self, x):
        x = self.flatten(x)
        out = self.mlp(x)
        return out

# train_loop is called once each epoch and trains model on training set
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation to update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print current training metrics for user
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        '''
        pred = pred.argmax(dim=1, keepdim=True)
        correct = pred.eq(y.view_as(pred)).sum().item()
        total_correct += correct
        total_loss += loss.item()
        #print(f"train loss: {loss.item():>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{NUM_TRAIN_BATCHES:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / NUM_TRAIN_BATCHES:>7f}   epoch avg train accuracy: {total_correct / (NUM_TRAIN_BATCHES * BATCH_SIZE):>7f}")
      '''

# test_loop evaluates model performance on test set with affine transformations
def test_loop():
  model.eval()  # put model in evalutation mode
  with torch.no_grad(): # tensors do not accumulate gradients since not training
    total_correct = 0
    # run each batch from the test set loader through the model
    for X, y in test_fourier_loader:
      X, y = X.to(DEVICE), y.to(DEVICE)

      # for a given batch, find the model's prediction out.argmax
      # then see if this prediction is correct or not (pred.eq = 1 or 0)
      out = model(X)
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    # finally print test accuracy for user
    accuracy = total_correct / (NUM_VAL_BATCHES * BATCH_SIZE)
    print(f"test accuracy with affine transformations: {accuracy:>7f}")

# eval_loop evaluates model performance on test set with no transformations
def eval_loop():
  model.eval()  # put model in evalutation mode
  with torch.no_grad(): # tensors do not accumulate gradients since not training
    total_correct = 0
    # run each batch from the eval set loader through the model
    for X, y in eval_fourier_loader:
      X, y = X.to(DEVICE), y.to(DEVICE)

      # for a given batch, find the model's prediction out.argmax
      # then see if this prediction is correct or not (pred.eq = 1 or 0)
      out = model(X)
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    # finally print eval accuracy for user
    accuracy = total_correct / (NUM_VAL_BATCHES * BATCH_SIZE)
    print(f"test accuracy: {accuracy:>7f}")



# transform MNIST images - take PIL image, return torch tensor of Fourier descriptors
def fourier_transform_train(img):
  img = np.asarray(img) # convert PIL image to numpy array for openCV
  ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) # binarize image
  contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # find outer contour of objects (digit) in image

  # since some images have artifacts disconnected from the digit, extract only
  # largest contour from the contour list (this should be the digit)
  largest_size = 0
  largest_index = 0
  for i, contour in enumerate(contours):
      if len(contour) > largest_size:
        largest_size = len(contour)
        largest_index = i
  contour = contours[largest_index]

  # use Pyefd to extract normalized Fourier descriptors then convert from numpy
  # array to torch tensor of dtype=float
  coeffs = pyefd.elliptic_fourier_descriptors(np.squeeze(contour), order=FOURIER_ORDER, normalize=True)
  return torch.from_numpy(coeffs).float()


# same as fourier_transform_train, but for test set -- adds affine transforms
def fourier_transform_test(img):
  img = np.asarray(img) # convert PIL image to numpy array for openCV
  ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)  # binarize image
  contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # find outer contour of objects (digit) in image

  # since some images have artifacts disconnected from the digit, extract only
  # largest contour from the contour list (this should be the digit)
  largest_size = 0
  largest_index = 0
  for i, contour in enumerate(contours):
      if len(contour) > largest_size:
        largest_size = len(contour)
        largest_index = i
  contour = contours[largest_index]

  # randomly select from 4 possible rotations and randomly select deltaX and
  # deltaY for translations in the range of (-3,3)
  rotations = [0, 90, 180, 270]
  angle = rotations[random.randint(0, len(rotations) - 1)]
  transY = random.randint(-3,3)
  transX = random.randint(-3,3)

  # linear algebra to apply randomly generated transformations to the digit contour
  theta = np.radians(angle)
  c, s = np.round(np.cos(theta)), np.round(np.sin(theta))
  R = np.array(((c, -s), (s, c)))
  trans = np.array((transX, transY))
  contourT = [np.expand_dims(np.transpose(np.dot(R, np.transpose(np.squeeze(contour)))) + trans, 1).astype(np.int32)]

  # use Pyefd to extract normalized Fourier descriptors then convert from numpy
  # array to torch tensor of dtype=float
  coeffs = pyefd.elliptic_fourier_descriptors(np.squeeze(contourT), order=FOURIER_ORDER, normalize=True)
  return torch.from_numpy(coeffs).float()


In [9]:
# test values for fourier series order in the range (1, 8)
for FOURIER_ORDER in range(1, 8):
  # seed RNGs
  torch.manual_seed(RAND_SEED)
  random.seed(RAND_SEED)

  # create train, eval, and test datasets
  train_fourier_data = datasets.MNIST(root=ROOT_DIR, train=True, download=True, transform=fourier_transform_train) # train data
  eval_fourier_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=fourier_transform_train) # test data WITHOUT affine transforms
  test_fourier_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=fourier_transform_test) # test data WITH affine transforms


  # create generator for dataloaders and create dataloaders for each dataset
  g = torch.Generator()
  g.manual_seed(RAND_SEED)
  train_fourier_loader = DataLoader(train_fourier_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
  eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
  test_fourier_loader = DataLoader(test_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)


  # initalize model object and load model parameters into optimizer
  model = LinearClassifierFourier()
  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  print("\n\n\n Fourier order is:"+str(FOURIER_ORDER)+"\n\n\n")  

  # train for EPOCHS number of epochs then evaluate on test data with affine transformations
  for i in range(EPOCHS):
      print("Epoch " + str(i + 1) + "\n")
      train_loop(dataloader=train_fourier_loader,model=model,loss_fn=loss_fn,optimizer=optimizer)
      eval_loop()
      print("\n-------------------------------\n")
  
  test_loop()
    




 Fourier order is:1



Epoch 1

loss: 2.302356  [    0/60000]
loss: 1.847657  [50000/60000]
test accuracy: 0.269900

-------------------------------

Epoch 2

loss: 1.804473  [    0/60000]
loss: 1.883050  [50000/60000]
test accuracy: 0.270300

-------------------------------

Epoch 3

loss: 1.854061  [    0/60000]
loss: 1.872818  [50000/60000]
test accuracy: 0.274800

-------------------------------

Epoch 4

loss: 1.861413  [    0/60000]
loss: 1.843984  [50000/60000]
test accuracy: 0.267400

-------------------------------

Epoch 5

loss: 1.907628  [    0/60000]
loss: 1.850693  [50000/60000]
test accuracy: 0.275100

-------------------------------

Epoch 6

loss: 1.792173  [    0/60000]
loss: 1.823873  [50000/60000]
test accuracy: 0.281200

-------------------------------

Epoch 7

loss: 1.842126  [    0/60000]
loss: 1.865112  [50000/60000]
test accuracy: 0.274200

-------------------------------

Epoch 8

loss: 1.842466  [    0/60000]
loss: 1.811571  [50000/60000]
test accuracy: 0

In [10]:
# test values for fourier series order in the range (1, 8)
for FOURIER_ORDER in range(8, 10):
  # seed RNGs
  torch.manual_seed(RAND_SEED)
  random.seed(RAND_SEED)

  # create train, eval, and test datasets
  train_fourier_data = datasets.MNIST(root=ROOT_DIR, train=True, download=True, transform=fourier_transform_train) # train data
  eval_fourier_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=fourier_transform_train) # test data WITHOUT affine transforms
  test_fourier_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=fourier_transform_test) # test data WITH affine transforms


  # create generator for dataloaders and create dataloaders for each dataset
  g = torch.Generator()
  g.manual_seed(RAND_SEED)
  train_fourier_loader = DataLoader(train_fourier_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
  eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
  test_fourier_loader = DataLoader(test_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)


  # initalize model object and load model parameters into optimizer
  model = LinearClassifierFourier()
  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  print("\n\n\n Fourier order is:"+str(FOURIER_ORDER)+"\n\n\n")  

  # train for EPOCHS number of epochs then evaluate on test data with affine transformations
  for i in range(EPOCHS):
      print("Epoch " + str(i + 1) + "\n")
      train_loop(dataloader=train_fourier_loader,model=model,loss_fn=loss_fn,optimizer=optimizer)
      eval_loop()
      print("\n-------------------------------\n")
  
  test_loop()
    




 Fourier order is:7



Epoch 1

loss: 2.300968  [    0/60000]
loss: 0.329676  [50000/60000]
test accuracy: 0.912300

-------------------------------

Epoch 2

loss: 0.317363  [    0/60000]
loss: 0.285217  [50000/60000]
test accuracy: 0.939700

-------------------------------

Epoch 3

loss: 0.243225  [    0/60000]
loss: 0.278820  [50000/60000]
test accuracy: 0.949000

-------------------------------

Epoch 4

loss: 0.182522  [    0/60000]
loss: 0.226933  [50000/60000]
test accuracy: 0.954300

-------------------------------

Epoch 5

loss: 0.206702  [    0/60000]
loss: 0.128830  [50000/60000]
test accuracy: 0.954900

-------------------------------

Epoch 6

loss: 0.098069  [    0/60000]
loss: 0.125930  [50000/60000]
test accuracy: 0.958900

-------------------------------

Epoch 7

loss: 0.097331  [    0/60000]
loss: 0.149273  [50000/60000]
test accuracy: 0.960900

-------------------------------

Epoch 8

loss: 0.071730  [    0/60000]
loss: 0.095113  [50000/60000]
test accuracy: 0

Section 2: Original images through linear classifier (to compare)
train model on MNIST 
without getting the fourier descriptors 
return the original image 

https://pytorch.org/vision/stable/transforms.html

In [11]:
# mlp taking mnist images
class LinearClassifierOriginal(nn.Module):
    def __init__(self):
        super(LinearClassifierOriginal, self).__init__()
        self.flatten = nn.Flatten()
        self.mlp = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,NUM_CLASSES))

    def forward(self, x):
        x = self.flatten(x)
        out = self.mlp(x)
        return out

# Define transformation(s) to be applied to dataset-
transforms_norm = transforms.Compose(
      [
          transforms.ToTensor(),
          transforms.Normalize(mean = (0.1307,), std = (0.3081,)), # MNIST mean and stdev
          
      ]
  )

# transform functions - take PIL image, return img as 1x28x28 torch tensor
# normalize each image by mean and sd (MNIST): https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457
# convert normalized image to torch tensor and return 
def original_transform_train(img):  
  # retrun normalized image
  return transforms_norm(img) 

# add rotations and translations at test time
# normalize each image by mean and sd (MNIST)
# apply random rotation and translations (use pytorch methods): https://pytorch.org/vision/stable/generated/torchvision.transforms.RandomRotation.html#torchvision.transforms.RandomRotation
# OR https://pytorch.org/vision/0.9/transforms.html#torchvision.transforms.functional.affine
# 
# return torch tensor
def original_transform_test(img):
    img = transforms_norm(img)
    angle=((random.random())*60)-30
    
    transY = random.randint(-3,3)
    transX = random.randint(-3,3)
    new_img = transforms.functional.affine(img,angle,[transX,transY],1,0) 
    return new_img
#...........

# train_loop is called once each epoch and trains model on training set
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation to update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print current training metrics for user
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# test_loop evaluates model performance on test set with affine transformations
def test_loop():
  model.eval()  # put model in evalutation mode
  with torch.no_grad(): # tensors do not accumulate gradients since not training
    total_correct = 0
    # run each batch from the test set loader through the model
    for X, y in test_original_loader:
      X, y = X.to(DEVICE), y.to(DEVICE)

      # for a given batch, find the model's prediction out.argmax
      # then see if this prediction is correct or not (pred.eq = 1 or 0)
      out = model(X)
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    # finally print test accuracy for user
    accuracy = total_correct / (NUM_VAL_BATCHES * BATCH_SIZE)
    print(f"test accuracy with affine transformations: {accuracy:>7f}")

# eval_loop evaluates model performance on test set with no transformations
def eval_loop():
  model.eval()  # put model in evalutation mode
  with torch.no_grad(): # tensors do not accumulate gradients since not training
    total_correct = 0
    # run each batch from the eval set loader through the model
    for X, y in eval_original_loader:
      X, y = X.to(DEVICE), y.to(DEVICE)

      # for a given batch, find the model's prediction out.argmax
      # then see if this prediction is correct or not (pred.eq = 1 or 0)
      out = model(X)
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    # finally print eval accuracy for user
    accuracy = total_correct / (NUM_VAL_BATCHES * BATCH_SIZE)
    print(f"test accuracy: {accuracy:>7f}")



In [12]:
# TODO: create original image dataset
# TODO: create generators for original dataset
# Env vars
torch.use_deterministic_algorithms(True) # to ensure reproduceability
ROOT_DIR = "/mnist"
RAND_SEED = 0
DEVICE = "cpu"

# training hyperparameters
NUM_CLASSES = 10
EPOCHS = 10
#FOURIER_ORDER = 10
LEARNING_RATE = 1e-3
BATCH_SIZE = 500
NUM_TRAIN_BATCHES = 60000 // BATCH_SIZE
NUM_VAL_BATCHES = 10000 // BATCH_SIZE
loss_fn = nn.CrossEntropyLoss()

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# create train, eval, and test datasets
train_original_data = datasets.MNIST(root=ROOT_DIR, train=True, download=True, transform=original_transform_train) # train data
eval_original_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=original_transform_train) # test data WITHOUT affine transforms
test_original_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=original_transform_test) # test data WITH affine transforms


# create generator for dataloaders and create dataloaders for each dataset
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_original_loader = DataLoader(train_original_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
eval_original_loader = DataLoader(eval_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_original_loader = DataLoader(test_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)


# initalize model object and load model parameters into optimizer
model = LinearClassifierOriginal()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
for i in range(EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_original_loader,model=model,loss_fn=loss_fn,optimizer=optimizer)
    eval_loop()
    print("\n-------------------------------\n")
  
test_loop() 

Epoch 1

loss: 2.300539  [    0/60000]
loss: 0.185107  [50000/60000]
test accuracy: 0.956700

-------------------------------

Epoch 2

loss: 0.121919  [    0/60000]
loss: 0.099608  [50000/60000]
test accuracy: 0.970300

-------------------------------

Epoch 3

loss: 0.063124  [    0/60000]
loss: 0.093923  [50000/60000]
test accuracy: 0.974500

-------------------------------

Epoch 4

loss: 0.071086  [    0/60000]
loss: 0.049891  [50000/60000]
test accuracy: 0.976600

-------------------------------

Epoch 5

loss: 0.055841  [    0/60000]
loss: 0.022989  [50000/60000]
test accuracy: 0.980000

-------------------------------

Epoch 6

loss: 0.020021  [    0/60000]
loss: 0.039589  [50000/60000]
test accuracy: 0.979200

-------------------------------

Epoch 7

loss: 0.020829  [    0/60000]
loss: 0.016611  [50000/60000]
test accuracy: 0.979200

-------------------------------

Epoch 8

loss: 0.013227  [    0/60000]
loss: 0.016865  [50000/60000]
test accuracy: 0.978600

-----------------

In [13]:
# TODO: create original image dataset
# TODO: create generators for original dataset
# Env vars
torch.use_deterministic_algorithms(True) # to ensure reproduceability
ROOT_DIR = "/mnist"
RAND_SEED = 0
DEVICE = "cpu"

# training hyperparameters
NUM_CLASSES = 10
EPOCHS = 10
#FOURIER_ORDER = 10
LEARNING_RATE = 1e-3
BATCH_SIZE = 500
NUM_TRAIN_BATCHES = 60000 // BATCH_SIZE
NUM_VAL_BATCHES = 10000 // BATCH_SIZE
loss_fn = nn.CrossEntropyLoss()

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# create train, eval, and test datasets
train_original_data = datasets.MNIST(root=ROOT_DIR, train=True, download=True, transform=original_transform_test) # train data
eval_original_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=original_transform_test) # test data WITHOUT affine transforms
test_original_data = datasets.MNIST(root=ROOT_DIR, train=False, download=True, transform=original_transform_train) # test data WITH affine transforms


# create generator for dataloaders and create dataloaders for each dataset
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_original_loader = DataLoader(train_original_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
eval_original_loader = DataLoader(eval_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_original_loader = DataLoader(test_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)


# initalize model object and load model parameters into optimizer
model = LinearClassifierOriginal()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
for i in range(EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_original_loader,model=model,loss_fn=loss_fn,optimizer=optimizer)
    eval_loop()
    print("\n-------------------------------\n")
  
test_loop() 

Epoch 1

loss: 2.297735  [    0/60000]
loss: 0.454847  [50000/60000]
test accuracy: 0.884000

-------------------------------

Epoch 2

loss: 0.355037  [    0/60000]
loss: 0.235952  [50000/60000]
test accuracy: 0.927200

-------------------------------

Epoch 3

loss: 0.255411  [    0/60000]
loss: 0.247383  [50000/60000]
test accuracy: 0.936300

-------------------------------

Epoch 4

loss: 0.205320  [    0/60000]
loss: 0.192373  [50000/60000]
test accuracy: 0.949700

-------------------------------

Epoch 5

loss: 0.229786  [    0/60000]
loss: 0.137371  [50000/60000]
test accuracy: 0.946300

-------------------------------

Epoch 6

loss: 0.166466  [    0/60000]
loss: 0.128476  [50000/60000]
test accuracy: 0.955800

-------------------------------

Epoch 7

loss: 0.151000  [    0/60000]
loss: 0.137463  [50000/60000]
test accuracy: 0.960500

-------------------------------

Epoch 8

loss: 0.140446  [    0/60000]
loss: 0.083567  [50000/60000]
test accuracy: 0.960000

-----------------